In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

In [7]:
# Random seed
np.random.seed(42)

In [8]:
# Load data
df = pd.read_csv('../data/BMED_data_v6.csv')
df = df[(df['exp'] == 1) | (df['exp'] == 2)]

In [11]:
# Initialize scalers
scaler_feature = StandardScaler()
columns_to_scale = [col for col in df.columns if col not in ['exp', 't']]
scaled_data = scaler_feature.fit_transform(df[columns_to_scale])
scaled_df = pd.DataFrame(scaled_data, columns=columns_to_scale)
scaled_df

,T,V,E,Ki,CF_LA,CA_LA,VF,VA
0,1.0,0.0,0.0,0.0,0.968636,-0.990628,0.619032,-0.327986
1,1.0,0.0,0.0,0.0,0.798203,-0.874278,0.916167,-0.630742
2,1.0,0.0,0.0,0.0,0.629015,-0.695278,0.916167,-0.630742
3,1.0,0.0,0.0,0.0,0.421883,-0.376273,0.321897,-0.327986
4,1.0,0.0,0.0,0.0,-0.148507,0.288585,-0.420942,-0.025230
5,1.0,0.0,0.0,0.0,-1.376990,1.541587,-0.718077,-0.933499
6,-1.0,0.0,0.0,0.0,0.968636,-0.990628,0.619032,-0.327986
7,-1.0,0.0,0.0,0.0,0.790117,-0.867246,0.916167,-0.327986
8,-1.0,0.0,0.0,0.0,0.115229,-0.128230,0.767600,-0.630742
9,-1.0,0.0,0.0,0.0,0.276953,-0.139098,-0.272374,-0.025230


In [10]:
# Split the data by experiment number
dict_spline = {}
for exp in df['exp'].unique():
    dict_spline[exp] = df[df['exp'] == exp].sort_values('t')
dict_spline

{np.int64(1):    exp      T   V     E   Ki    t   CF_LA   CA_LA    VF    VA
 0    1  25.06  20  0.25  1.0  0.0  0.5000  0.0000  1.00  1.00
 1    1  25.06  20  0.25  1.0  1.0  0.4726  0.0182  1.02  0.99
 2    1  25.06  20  0.25  1.0  2.0  0.4454  0.0462  1.02  0.99
 3    1  25.06  20  0.25  1.0  3.0  0.4121  0.0961  0.98  1.00
 4    1  25.06  20  0.25  1.0  4.0  0.3204  0.2001  0.93  1.01
 5    1  25.06  20  0.25  1.0  5.1  0.1229  0.3961  0.91  0.98,
 np.int64(2):     exp      T   V     E   Ki    t   CF_LA   CA_LA    VF    VA
 6     2  23.54  20  0.25  1.0  0.0  0.5000  0.0000  1.00  1.00
 7     2  23.54  20  0.25  1.0  1.0  0.4713  0.0193  1.02  1.00
 8     2  23.54  20  0.25  1.0  3.0  0.3628  0.1349  1.01  0.99
 9     2  23.54  20  0.25  1.0  4.0  0.3888  0.1332  0.94  1.01
 10    2  23.54  20  0.25  1.0  5.0  0.1350  0.3609  0.87  1.06
 11    2  23.54  20  0.25  1.0  6.0  0.0000  0.4545  0.80  1.10}